In [271]:
import numpy as np
import lightgbm as lgb
import os

In [272]:
gacrp_home=os.environ['GACRP_HOME']

In [273]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import gc
import time
from pandas.core.common import SettingWithCopyWarning
import warnings
import lightgbm as lgb
from sklearn.model_selection import GroupKFold

In [274]:
warnings.simplefilter('error', SettingWithCopyWarning)
gc.enable()
%matplotlib inline

In [275]:
train= pd.read_csv(gacrp_home+"/dat/train_flat_100.csv"
                   ,dtype={'date':str,'fullVisitorId':str,'sessionId':str,'adContent':str})

In [276]:
print(train.shape)

(99, 55)


In [277]:
def get_folds(df=None,n_splits=5):
    unique_vis = np.array(sorted(df['fullVisitorId'].unique()))
    print(len(unique_vis))
    folds = GroupKFold(n_splits=n_splits)
    fold_ids=[]
    #idsはindexっぽい
    ids = np.arange(df.shape[0])
    for trn_vis,val_vis in folds.split(X=unique_vis,y=unique_vis,groups=unique_vis):
        fold_ids.append(
            [
                ids[df['fullVisitorId'].isin(unique_vis[trn_vis])],
                ids[df['fullVisitorId'].isin(unique_vis[val_vis])]
            ]
        )
    #print(len(fold_ids))
    return fold_ids
        

In [278]:
# transactionRevenueは教師側に持っていく
y_reg=train['transactionRevenue'].fillna(0)
del train['transactionRevenue']


In [279]:
for df in [train]:
    df['date']=pd.to_datetime(df['visitStartTime'],unit='s')
    df['sess_date_dow']=df['date'].dt.dayofweek
    df['sess_date_hours']=df['date'].dt.hour
    df['sess_date_dom']=df['date'].dt.day

In [280]:
excluded_features=[
    'date','fullVisitorId','sessionId','transactionRevenue','visitId','visitStartTime',
    'adwordsClickInfo.adNetworkType','browserSize','browserVersion','campaignCode','cityId','adwordsClickInfo.criteriaParameters','flashVersion','adwordsClickInfo.gclId','adwordsClickInfo.isVideoAd','language','latitude','longitude','mobileDeviceBranding','mobileDeviceInfo','mobileDeviceMarketingName','mobileDeviceModel','mobileInputSelector','networkLocation','operatingSystemVersion','adwordsClickInfo.page','screenColors','screenResolution','adwordsClickInfo.slot','socialEngagementType','visits'
]
categorical_features=[
    _f for _f in train.columns
    if (_f not in excluded_features) & (train[_f].dtype =='object')
]
print(categorical_features)

['channelGrouping', 'browser', 'operatingSystem', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'networkDomain', 'campaign', 'source', 'medium', 'keyword', 'isTrueDirect', 'referralPath', 'adContent']


In [281]:
for f in categorical_features:
    train[f],indexer = pd.factorize(train[f])
    

In [282]:
#print(train['continent'][0:100])

In [283]:
# prediction
folds = get_folds(df=train,n_splits=5)
#foldsは分割データのインデックスの配列（2,3,5,7…とか。）
#print("folds:",folds)
train_features=[_f for _f in train.columns if _f not in excluded_features]
print(train_features)
print(len(train_features))

98
['channelGrouping', 'visitNumber', 'browser', 'operatingSystem', 'isMobile', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'networkDomain', 'hits', 'pageviews', 'bounces', 'newVisits', 'campaign', 'source', 'medium', 'keyword', 'isTrueDirect', 'referralPath', 'adContent', 'sess_date_dow', 'sess_date_hours', 'sess_date_dom']
27


In [284]:
importances = pd.DataFrame()
oof_reg_preds = np.zeros(train.shape[0])

In [286]:
for fold_ , (trn_,val_)in enumerate(folds):
    trn_x,trn_y = train[train_features].iloc[trn_],y_reg.iloc[trn_]
    val_x,val_y = train[train_features].iloc[val_],y_reg.iloc[val_]
    
    reg= lgb.LGBMRegressor(
        num_leaves=31,
        learning_rage=0.03,
        n_estimators=1000,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=1
    )
    reg.fit(
        trn_x,np.log1p(trn_y),
        eval_set=[(val_x,np.log1p(val_y))],
        early_stopping_rounds=50,
        verbose=100,
        eval_metric='rmse'
    )

TypeError: Cannot initialize Dataset from DataFrame